In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import ast
import re
from bs4 import BeautifulSoup

In [2]:
# Read the scraped data from the CSV file into a DataFrame
df = pd.read_csv('../data/raw/leetcode_latest.csv')
dff = df.copy()

In [3]:
# Display a random sample of 10 rows from the DataFrame
df.sample(10)

,internal_id,frontend_id,title,titleSlug,difficulty,is_premium,topic_tags,similar_questions,no_similar_questions,acceptance,accepted,submission,discussion_count,likes,dislikes,description,problem_URL,solution_URL,last_updated
1548,1688,1549,The Most Recent Orders for Each Product,the-most-recent-orders-for-each-product,Medium,True,['Database'],"['The Most Recent Three Orders', 'The Most Fre...",2,64.921021,37401,57610,8,139,12,NaN,https://leetcode.com/problems/the-most-recent-...,NaN,2025-10-26
935,973,936,Stamping The Sequence,stamping-the-sequence,Hard,False,"['String', 'Stack', 'Greedy', 'Queue']",[],0,62.042692,62781,101190,19,1571,222,<p>You are given two strings <code>stamp</code...,https://leetcode.com/problems/stamping-the-seq...,https://leetcode.com/problems/stamping-the-seq...,2025-10-26
2794,2898,2795,Parallel Execution of Promises for Individual ...,parallel-execution-of-promises-for-individual-...,Medium,True,[],[],0,90.268767,974,1079,2,15,5,NaN,https://leetcode.com/problems/parallel-executi...,NaN,2025-10-26
1781,1891,1782,Count Pairs Of Nodes,count-pairs-of-nodes,Hard,False,"['Array', 'Hash Table', 'Two Pointers', 'Binar...",[],0,41.654805,9072,21779,7,338,171,<p>You are given an undirected graph defined b...,https://leetcode.com/problems/count-pairs-of-n...,https://leetcode.com/problems/count-pairs-of-n...,2025-10-26
2040,2185,2041,Accepted Candidates From the Interviews,accepted-candidates-from-the-interviews,Medium,True,['Database'],[],0,78.662371,15136,19242,8,58,28,NaN,https://leetcode.com/problems/accepted-candida...,NaN,2025-10-26
3399,3740,3400,Maximum Number of Matching Indices After Right...,maximum-number-of-matching-indices-after-right...,Medium,True,"['Array', 'Two Pointers', 'Simulation']",['Rotate Array'],1,84.513007,1397,1653,2,14,1,NaN,https://leetcode.com/problems/maximum-number-o...,NaN,2025-10-26
3029,3272,3030,Find the Grid of Region Average,find-the-grid-of-region-average,Medium,False,"['Array', 'Matrix']","['Range Sum Query 2D - Immutable', 'K Radius S...",2,42.814126,11517,26900,23,88,134,<p>You are given <code>m x n</code> grid <code...,https://leetcode.com/problems/find-the-grid-of...,https://leetcode.com/problems/find-the-grid-of...,2025-10-26
164,165,165,Compare Version Numbers,compare-version-numbers,Medium,False,"['Two Pointers', 'String']",[],0,45.699000,674706,1476412,291,3118,2791,"<p>Given two <strong>version strings</strong>,...",https://leetcode.com/problems/compare-version-...,https://leetcode.com/problems/compare-version-...,2025-10-26
709,894,710,Random Pick with Blacklist,random-pick-with-blacklist,Hard,False,"['Array', 'Hash Table', 'Math', 'Binary Search...","['Random Pick Index', 'Random Pick with Weight...",3,34.300980,49946,145611,16,892,121,<p>You are given an integer <code>n</code> and...,https://leetcode.com/problems/random-pick-with...,https://leetcode.com/problems/random-pick-with...,2025-10-26
467,468,468,Validate IP Address,validate-ip-address,Medium,False,['String'],"['IP to CIDR', 'Strong Password Checker II']",2,27.994837,193044,689570,58,1097,2742,"<p>Given a string <code>queryIP</code>, return...",https://leetcode.com/problems/validate-ip-addr...,https://leetcode.com/problems/validate-ip-addr...,2025-10-26


In [4]:
# Display concise information about the DataFrame, including the data types and memory usage
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3730 entries, 0 to 3729
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   internal_id           3730 non-null   int64  
 1   frontend_id           3730 non-null   int64  
 2   title                 3730 non-null   object 
 3   titleSlug             3730 non-null   object 
 4   difficulty            3730 non-null   object 
 5   is_premium            3730 non-null   bool   
 6   topic_tags            3730 non-null   object 
 7   similar_questions     3730 non-null   object 
 8   no_similar_questions  3730 non-null   int64  
 9   acceptance            3730 non-null   float64
 10  accepted              3730 non-null   int64  
 11  submission            3730 non-null   int64  
 12  discussion_count      3730 non-null   int64  
 13  likes                 3730 non-null   int64  
 14  dislikes              3730 non-null   int64  
 15  description          

In [5]:
# Count the number of duplicated rows in the DataFrame
df.duplicated().sum()

np.int64(0)

In [6]:
# Count the number of missing values in each column of the DataFrame
df.isnull().sum()

internal_id               0
frontend_id               0
title                     0
titleSlug                 0
difficulty                0
is_premium                0
topic_tags                0
similar_questions         0
no_similar_questions      0
acceptance                0
accepted                  0
submission                0
discussion_count          0
likes                     0
dislikes                  0
description             739
problem_URL               0
solution_URL            739
last_updated              0
dtype: int64

In [7]:
# Get the unique values in the `is_premium` column.
df['is_premium'].unique()

array([False,  True])

In [8]:
#  Check the number of missing values in the `is_premium` column.
df['is_premium'].isnull().sum()

np.int64(0)

In [9]:
# Replace all missing values in the `is_premium` column with `True`.
df['is_premium'].fillna(True, inplace=True)

In [10]:
# Get the number of premium and non-premium questions in the DataFrame.
df['is_premium'].value_counts()

is_premium
False    2991
True      739
Name: count, dtype: int64

In [11]:
# Get the data type of the 'is_premium' column.
df['is_premium'].dtype

dtype('bool')

In [12]:
#  Check the number of missing values in the `topic_tags` column.
df['topic_tags'].isnull().sum()

np.int64(0)

In [13]:
# The `selected_ids` list contains a set of LeetCode question IDs
# that are all related to JavaScript
# Number of `selected_ids` is 37
selected_ids = [2693, 2694, 2695, 2703, 2704, 2705, 2715, 2721, 2722, 2723, 2724,
2725, 2726, 2727, 2618, 2619, 2620, 2621, 2622, 2623, 2624, 2625, 2626,
2627, 2629, 2630, 2631, 2634, 2635, 2637, 2648, 2649, 2650, 2665, 2666, 2667, 2677]

In [14]:
# Assign "JavaScript" topic tag to rows where frontend_id is in selected_ids
df.loc[df['frontend_id'].astype(int).isin(selected_ids), 'topic_tags'] = "JavaScript"

In [15]:
# The `selected_ids` list contains a set of LeetCode question IDs
# that are all related to Pandas
# Number of `selected_ids` is 15
selected_ids = [2877, 2878, 2879, 2880, 2881, 2882, 2883, 2884, 2885, 2886, 2887,
2888, 2889, 2890, 2891]

In [17]:
# Assign "Pandas" topic tag to rows where frontend_id is in selected_ids
df.loc[df['frontend_id'].astype(int).isin(selected_ids), 'topic_tags'] = "Pandas"

In [18]:
#  Check the number of missing values in the `topic_tags` column.
df['topic_tags'].isnull().sum()

np.int64(0)

In [19]:
# Count the number of null values in the 'similar_questions'
df['similar_questions'].isnull().sum()

np.int64(0)

In [20]:
# Set an empty string for 'similar_questions' where 'is_premium' is False and 'similar_questions' is null
df.loc[(df['is_premium'] == False) & df['similar_questions'].isnull(), 'similar_questions'] = ''

In [21]:
# Count the number of null values in the 'similar_questions'
df['similar_questions'].isnull().sum()

np.int64(0)

In [22]:
# Count the number of null values in each column
df.isnull().sum()

internal_id               0
frontend_id               0
title                     0
titleSlug                 0
difficulty                0
is_premium                0
topic_tags                0
similar_questions         0
no_similar_questions      0
acceptance                0
accepted                  0
submission                0
discussion_count          0
likes                     0
dislikes                  0
description             739
problem_URL               0
solution_URL            739
last_updated              0
dtype: int64

In [23]:
# Retrieve the 'title' column
df['title']

0                                                 Two Sum
1                                         Add Two Numbers
2          Longest Substring Without Repeating Characters
3                             Median of Two Sorted Arrays
4                           Longest Palindromic Substring
                              ...                        
3725               Remove Zeros in Decimal Representation
3726                   Maximum Alternating Sum of Squares
3727    Stable Subarrays With Equal Boundary and Inter...
3728    Count Distinct Subarrays Divisible by K in Sor...
3729                    Maximum Calories Burnt from Jumps
Name: title, Length: 3730, dtype: object

In [24]:
# Use the numeric 'frontend_id' directly instead of parsing from title
df['frontend_id'] = df['frontend_id'].astype(int)

In [25]:
# Assign 'frontend_id' as 'id' directly since it's already numeric
df['id'] = df['frontend_id'].astype(int)

In [26]:
df['page_number'] = pd.cut(df['id'], bins=range(1, 3002, 50), include_lowest=True, right=False).apply(lambda x: (x.left//50)+1)

In [27]:
# Retrieve the 'similar_questions' column
df['similar_questions']

0       ['3Sum', '4Sum', 'Two Sum II - Input Array Is ...
1       ['Multiply Strings', 'Add Binary', 'Sum of Two...
2       ['Longest Substring with At Most Two Distinct ...
3                  ['Median of a Row Wise Sorted Matrix']
4       ['Shortest Palindrome', 'Palindrome Permutatio...
                              ...                        
3725                                                   []
3726                                                   []
3727                                                   []
3728                                                   []
3729                                                   []
Name: similar_questions, Length: 3730, dtype: object

In [28]:
# Performing string manipulation on the 'similar_questions' column in the DataFrame 'df':
# Removing single quotes and then splitting the string into a list using ', ' as the delimiter.
df['similar_questions'].str.split(", ")

0       [['3Sum', '4Sum', 'Two Sum II - Input Array Is...
1       [['Multiply Strings', 'Add Binary', 'Sum of Tw...
2       [['Longest Substring with At Most Two Distinct...
3                [['Median of a Row Wise Sorted Matrix']]
4       [['Shortest Palindrome', 'Palindrome Permutati...
                              ...                        
3725                                                 [[]]
3726                                                 [[]]
3727                                                 [[]]
3728                                                 [[]]
3729                                                 [[]]
Name: similar_questions, Length: 3730, dtype: object

In [29]:
# Removing single quotes from the 'similar_questions' column in the DataFrame 'df'
# and then splitting the string into a list using ', ' as the delimiter.
df['similar_questions'] = df['similar_questions'].str.split(", ")

In [30]:
# Applying a lambda function to the 'similar_questions' column in the DataFrame 'df':
# Counting the number of elements in each list (if it is a list and the first element is not an empty string), otherwise returning 0.
df['similar_questions'].apply(lambda x: len(x) if isinstance(x,list) and x[0]!='' else 0)

0       21
1        8
2       11
3        1
4        6
        ..
3725     1
3726     1
3727     1
3728     1
3729     1
Name: similar_questions, Length: 3730, dtype: int64

In [31]:
# Creating a new column 'no_similar_questions' in the DataFrame 'df':
# Assigning the count of elements in each list from the 'similar_questions' column, or 0 if it's not a list.
df['no_similar_questions'] = df['similar_questions'].apply(lambda x: len(x) if isinstance(x,list) and x!=[''] else 0)

In [32]:
# Setting 'no_similar_questions' to pd.NA for rows where 'is_premium' is True
# and 'no_similar_questions' is 0.
df.loc[(df['is_premium'] == True) & (df['no_similar_questions'] == 0), 'no_similar_questions'] = pd.NA

In [33]:
# Retrieve specific columns ['acceptance', 'accepted', 'submission', 'solution', 'discussion_count', 'likes', 'dislikes']
# from the DataFrame 'df'
df[['acceptance','accepted','submission','discussion_count','likes','dislikes']]

,acceptance,accepted,submission,discussion_count,likes,dislikes
0,56.472280,19293695,34164957,1646,65123,2412
1,47.193252,6347474,13449996,1000,35167,7025
2,37.792455,8362837,22128327,720,43432,2124
3,45.042468,3791262,8417111,821,31141,3496
4,36.650205,4254554,11608573,530,31727,1952
...,...,...,...,...,...,...
3725,75.995837,26926,35434,7,20,1
3726,55.893230,18668,33420,7,22,0
3727,20.696363,7291,35346,30,56,1
3728,19.486316,2306,11842,7,17,0


In [34]:
# Displaying information about selected columns ['acceptance', 'accepted', 'submission', 'solution', 'discussion_count', 'likes', 'dislikes']
# in the DataFrame 'df'.
df[['acceptance','accepted','submission','discussion_count','likes','dislikes']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3730 entries, 0 to 3729
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acceptance        3730 non-null   float64
 1   accepted          3730 non-null   int64  
 2   submission        3730 non-null   int64  
 3   discussion_count  3730 non-null   int64  
 4   likes             3730 non-null   int64  
 5   dislikes          3730 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 175.0 KB


In [35]:
# Stripping the percentage sign from the 'acceptance' column in the DataFrame 'df'
# and converting the values to floating-point numbers.
df['acceptance'] = pd.to_numeric(df['acceptance'], errors='coerce')

In [36]:
# Measures how well-received a problem is based on likes vs dislikes
df['likebility'] = (
    (df['likes'] / (df['likes'] + df['dislikes'] + 1)) * 100
).round(2)

In [37]:
# Helper function to handle both numeric and string values ---
def convert_km_to_int(series):
    """
    Converts columns like '1.2K' or '1.1M' to integers.
    Works safely even if column is already numeric.
    """
    if series.dtype == 'O':  # object dtype => maybe strings
        series = (
            series.astype(str)
            .str.replace('K', 'e3', regex=False)
            .str.replace('M', 'e6', regex=False)
        )
    return pd.to_numeric(series, errors='coerce').astype('Int32')

df['acceptance'] = pd.to_numeric(df['acceptance'], errors='coerce')
df['accepted'] = convert_km_to_int(df['accepted'])
df['submission'] = convert_km_to_int(df['submission'])
df['discussion_count'] = convert_km_to_int(df['discussion_count'])
df['likes'] = convert_km_to_int(df['likes'])
df['dislikes'] = convert_km_to_int(df['dislikes'])

# --- Derived metrics ---
df['acceptance_rate'] = (df['accepted'] / (df['submission'] + 1) * 100).round(2)
df['like_ratio'] = (df['likes'] / (df['likes'] + df['dislikes'] + 1)).round(2)
df['num_tags'] = df['topic_tags'].apply(len)
df['is_popular'] = df['likes'] > df['likes'].median()
df['likebility'] = (df['like_ratio'] * 100).round(2)

In [38]:
# Displaying selected key numeric and derived columns from the DataFrame 'df'
df[['acceptance', 'accepted', 'submission', 'discussion_count',
    'likes', 'dislikes', 'acceptance_rate', 'like_ratio', 'likebility']].head()

,acceptance,accepted,submission,discussion_count,likes,dislikes,acceptance_rate,like_ratio,likebility
0,56.472280,19293695,34164957,1646,65123,2412,56.47,0.96,96.0
1,47.193252,6347474,13449996,1000,35167,7025,47.19,0.83,83.0
2,37.792455,8362837,22128327,720,43432,2124,37.79,0.95,95.0
3,45.042468,3791262,8417111,821,31141,3496,45.04,0.9,90.0
4,36.650205,4254554,11608573,530,31727,1952,36.65,0.94,94.0


In [39]:
# Displaying the shape (number of rows and columns) of the DataFrame 'df'
df.shape

(3730, 26)

In [40]:
# Arrangement of columns (updated for latest dataset)
column_order = [
    'page_number',
    'frontend_id',
    'title',
    'titleSlug',
    'difficulty',
    'is_premium',
    'topic_tags',
    'num_tags',
    'similar_questions',
    'no_similar_questions',
    'acceptance',
    'accepted',
    'submission',
    'acceptance_rate',
    'discussion_count',
    'likes',
    'dislikes',
    'like_ratio',
    'likebility',
    'is_popular',
    'problem_URL',
    'solution_URL',
]

In [41]:
# Check number of columns
len(column_order)

22

In [42]:
# Column Rearrangement
df = df[column_order]

In [43]:
# Information about dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3730 entries, 0 to 3729
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   page_number           3000 non-null   category
 1   frontend_id           3730 non-null   int64   
 2   title                 3730 non-null   object  
 3   titleSlug             3730 non-null   object  
 4   difficulty            3730 non-null   object  
 5   is_premium            3730 non-null   bool    
 6   topic_tags            3730 non-null   object  
 7   num_tags              3730 non-null   int64   
 8   similar_questions     3730 non-null   object  
 9   no_similar_questions  3730 non-null   float64 
 10  acceptance            3730 non-null   float64 
 11  accepted              3730 non-null   Int32   
 12  submission            3730 non-null   Int32   
 13  acceptance_rate       3730 non-null   Float64 
 14  discussion_count      3730 non-null   Int32   
 15  like

In [47]:
# Fix nulls in 'page_number' and 'solution_URL'
max_id = df['frontend_id'].max() + 50
df['page_number'] = pd.cut(
    df['frontend_id'],
    bins=range(1, max_id, 50),
    include_lowest=True,
    right=False
).apply(lambda x: (x.left // 50) + 1)

# Leave missing solution URLs blank (not NaN)
df['solution_URL'].fillna('', inplace=True)

In [48]:
# Displaying the count of null values for columns with non-zero null counts
# in the DataFrame 'df'

df.isnull().sum()[df.isnull().sum()!=0]

Series([], dtype: int64)

In [49]:
# Saving the preprocessed DataFrame 'df'
# to a CSV file named 'preprocessed_data.csv' without including the index.
df.to_csv('preprocessed_data.csv', index=None)

In [50]:
pd.read_csv('../data/processed/preprocessed_data.csv').info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3730 entries, 0 to 3729
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   page_number           3730 non-null   int64  
 1   frontend_id           3730 non-null   int64  
 2   title                 3730 non-null   object 
 3   titleSlug             3730 non-null   object 
 4   difficulty            3730 non-null   object 
 5   is_premium            3730 non-null   bool   
 6   topic_tags            3730 non-null   object 
 7   num_tags              3730 non-null   int64  
 8   similar_questions     3730 non-null   object 
 9   no_similar_questions  3730 non-null   float64
 10  acceptance            3730 non-null   float64
 11  accepted              3730 non-null   int64  
 12  submission            3730 non-null   int64  
 13  acceptance_rate       3730 non-null   float64
 14  discussion_count      3730 non-null   int64  
 15  likes                